# Praktikum Week 8 Data Acquisition

## Muhammad Naufal Ilman
## 162012133067
## Link Github: 

## 1. Dengan menggunakan Python Requests dan BeautifulSoup cobalah untuk scraping 1 halaman website unair news (https://unair.ac.id/news) dan judul berita yang ada hari ini

In [5]:
# import library yang diperlukan
import requests
from bs4 import BeautifulSoup

In [11]:
# halaman berita unair yang akan di scraping
url = "https://unair.ac.id/category/berita/"
# mengirim permintaan HTTP GET ke url
response = requests.get(url)
# mengambil response dalam bentuk format HTML, kemudian disimpan dalam variabel 
rawhtml = response.text
# melakukan parsing pada HTML
soup = BeautifulSoup(rawhtml, 'html.parser')
for i in soup.find_all('h3'):
    print(i.get_text())



				Websvaganza 2023 Hadirkan Bazar Kosmetik untuk Dorong Kepercayaan Diri dan Hilangkan Insecure			



				Mahasiswa UNAIR Sabet Juara 1 Kategori Lomba Infografis KOMINFO Jawa Timur			



				Pentingnya Mengenal Potensi Diri Melalui Pemahaman Emosional			



				Dukung Pendidikan Merata UNAIR Berikan Beasiswa Siswa Sekolah Dasar			



				Apa Sih UKM Wanala ?			



				Delegasi FH Kembali Juarai Lomba Debat Diponegoro Law Fair			



				Mahasiswa FH UNAIR Sabet Juara III Lomba Paper Internasional			



				Pakar Hukum Humaniter Internasional UNAIR Sebut Tindakan Israel Merupakan Crime Against Humanity			



				Ancam Boikot SpaceX, Dosen UNAIR Sebut Israel Tidak Ingin Aksi Genosida Diketahui Dunia Luar			



				Kunjungan Resmi Wufeng University, Sepakati Berbagai MoU dengan UNAIR			



				Dr Andriyanto, Alumnus UNAIR yang Dilantik Menjadi Pj Bupati Pasuruan			



				Taiwan Higher Education Fair Tawarkan Beragam Beasiswa Kuliah			



## 2.	Lalu, lakukan crawling unair news dengan mengkombinasikan Python Request, beautifulsoup, dan for loop, untuk mengambil judul berita dari kategori featured news

In [14]:
# import libary yang diperlukan
import requests
from bs4 import BeautifulSoup
import json

In [17]:
# Define the initial URL
base_url = 'https://unair.ac.id/category/featured/'
next_page = base_url

# Create a list to store the results
results = []

# Loop until there are no more "next" links
while next_page:
    # Send an HTTP GET request to the current page
    response = requests.get(next_page)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the data you need from the current page
        # Replace this with your specific scraping logic
        # For example, if the titles are inside <h3> elements:
        titles = soup.find_all('h3')
        for title in titles:
            title_text = title.get_text().strip()  # Remove leading/trailing whitespace and newline characters
            results.append(title_text)

        # Find the link to the next page
        next_page_tag = soup.find('link', rel='next')
        if next_page_tag:
            next_page = next_page_tag.get('href')
        else:
            next_page = None  # No more "next" link

# Save the results to a JSON file
with open('scraped_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

print("Data has been saved to scraped_data.json")


Data has been saved to scraped_data.json


## 3. Setelah itu, gunakan scrapy untuk melakukan crawling website https://bit.ly/scrapingtry dan menyimpan judul game beserta harganya dalam tabel

link website: https://store.playstation.com/en-id/category/05a2d027-cedc-4ac0-abeb-8fc26fec7180/

In [28]:
import scrapy
import csv
from scrapy.spiders import Spider
class GamePricesSpider(scrapy.Spider):
    name = 'game_prices'
    start_urls = ['https://store.playstation.com/en-id/category/05a2d027-cedc-4ac0-abeb-8fc26fec7180/1']

    def __init__(self, max_pages=16, *args, **kwargs):
        super(GamePricesSpider, self).__init__(*args, **kwargs)
        self.max_pages = max_pages

    def parse(self, response):
        games = response.css('.psw-t-body.psw-c-t-1.psw-t-truncate-2.psw-m-b-2')
        prices = response.css('.psw-m-r-3')

        # Open the CSV file with proper encoding
        with open('scraped_data.csv', 'a', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            if response.url.endswith('/1'):
                writer.writerow(['title', 'price'])  # Write the header
            for game, price in zip(games, prices):
                title = game.css('span::text').get()
                game_price = price.css('span::text').get()
                # Remove the non-breaking space (U+00A0) characters
                title = title.replace(u'\u00A0', u'')
                game_price = game_price.replace(u'\u00A0', u'')
                writer.writerow([title, game_price])

        # Generate the next page URL based on the pattern
        self.page_number = int(response.url.split('/')[-1]) + 1
        next_page_url = f'https://store.playstation.com/en-id/category/05a2d027-cedc-4ac0-abeb-8fc26fec7180/{self.page_number}'

        # Follow the next page URL
        if self.page_number <= self.max_pages:
            yield response.follow(next_page_url, callback=self.parse)
